In [6]:
import numpy as np
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
import pylab as plt #import Matplotlib plotting interface
import math
import community as community_louvain
import collections

In [5]:
class address:
    
    def __init__(self, coins, public):
        
        self.public = public
        self.coins = coins
       
    def receive(self, amount):
        self.coins += amount
        
    def send(self, other, amount):
        
        if(self.coins >= amount):
            other.receive(amount)
            self.coins -= amount
            amount_sent = amount
        else:
            
            other.receive(self.coins)
            self.coins = 0
            amount_sent = self.coins 
            
        return amount_sent

class transaction:
        
    def __init__(self, sums_in, sums_out):

        self.sums_in = sums_in
        self.sums_out = sums_out
        
        
class blockchain:
    
    def __init__(self, number_of_users, coins, num_shops, mixing_probability, number_of_mixers):
        
        self.num_shops = num_shops
        self.network = np.zeros((number_of_users + number_of_mixers, number_of_users + number_of_mixers))
        self.addresses = []
        self.transactions = []
        for i in range(number_of_users):
            self.addresses.append(user(mixing_probability, number_of_users, coins, i))
        self.num_users = len(self.addresses)
        i = number_of_users
        while(i < number_of_users + number_of_mixers):
            
            self.addresses.append(address(0, i))
            i = i + 1
            
    def generate_address(self):
        self.addresses.append(address(coins, len(self.addresses)))
        
    def size(self):
        return len(self.addresses)
            
    def update_network(self, address_send, address_receive, amount):
        self.network[address_send][address_receive] += amount
    
    def print_ledger(self):
        
        for address in self.addresses:
            print("amount: " + str(address.coins) + " id:" + str(address.public))
    """     
    def execute_transactions(self, transactions_list):
        sums_in = np.zeros(len(self.addresses))
        sums_out = np.zeros(len(self.addresses))
        for addresses_to_send in transactions_list:
            
            for i, address in enumerate(self.addresses):
                if(addresses_to_send[i] != i and i > self.num_shops -1):
                    amount_sent = address.send(self.addresses[addresses_to_send[i]], 1)
                    sums_in[i] += amount_sent
                    sums_out[addresses_to_send[i]] += amount_sent
                    self.update_network(i, addresses_to_send[i], amount_sent)
            
        self.transactions.append(transaction(sums_in, sums_out))
    """     
    
    def execute_transactions(self, addresses_in, addresses_out, sums):
        sums_in = np.zeros(len(self.addresses))
        sums_out = np.zeros(len(self.addresses))
        
        for i in range(len(addresses_in)):
            address_in = self.addresses[addresses_in[i]]
            address_out = self.addresses[addresses_out[i]]
            sum_to_send = sums[i]
            amount_sent = address_in.send(address_out, sum_to_send)
            sums_in[addresses_in[i]] += amount_sent
            sums_out[addresses_out[i]] += amount_sent
            self.update_network(addresses_in[i], addresses_out[i], amount_sent)
            
        self.transactions.append(transaction(sums_in, sums_out))
        
        
class simulation:
    
    def __init__(self, size, coins, num_shops, mixing_probability, number_of_steps):
        self.blockchain  = blockchain(size, coins, num_shops, mixing_probability, number_of_steps)
        self.mixer = mixer(size, number_of_steps)
        self.number_of_steps = number_of_steps
        self.num_shops = num_shops
        
    
    def generate_sending_probabilities(self):
        
        probabilities = []
        for i in range(self.blockchain.size()):
            if(i < self.num_shops):
                probabilities.append(1/self.blockchain.size() * 3)
            else:
                probabilities.append((1 - (1/self.blockchain.size() * 3) * self.num_shops)/ (self.blockchain.size() - self.num_shops))
        return probabilities
    
    def generate_transactions_network(self):
        size = self.blockchain.size() + len(self.blockchain.transactions)
        self.transactions_network = np.zeros((size, size))
        
        for i, transaction in enumerate(self.blockchain.transactions):
            for j in range(len(transaction.sums_in)):
                self.transactions_network[j][self.blockchain.size() + i] += transaction.sums_in[j]
                self.transactions_network[self.blockchain.size() + i][j] += transaction.sums_out[j]
                

        return self.transactions_network
    
    def step(self):   
        number_of_transactions = random.randint(1, 2)
        transactions = []
        for i in range(number_of_transactions):
            probabilities = self.generate_sending_probabilities()
            addresses_to_send = np.random.choice(self.blockchain.size(), self.blockchain.size(), probabilities) 
            transactions.append(addresses_to_send)
        self.blockchain.execute_transactions(transactions)
        
    def step_with_mixer(self):
        transaction_to_perform = [self.blockchain.size()]
        for i in range(self.blockchain.num_users):
            mix, address, amount = self.blockchain.addresses[i].generate_transfer(self.blockchain.size())
            if(mix == 0):
                self.blockchain.execute_transactions([i], [address], [amount])
            else:
                self.blockchain.addresses[self.blockchain.size() - 1].add_to_mixng_pool(i, address, amount)
                self.blockchain.execute_transactions([i], [self.blockchain.size() - 1], [amount])
        addresses_out, sums_out = self.blockchain.addresses[self.blockchain.size() - 1].send_out()
        addresses_in = []
        for i in range(len(addresses_out)):
            addresses_in.append(self.blockchain.size() - 1)
        self.blockchain.execute_transactions(addresses_in, addresses_out, sums_out)
        
    def step_with_multiple_addresses(self):
        
        #transaction_to_perform = [self.blockchain.size()]
        
        for i in range(self.blockchain.num_users):
            mix, address, amount = self.blockchain.addresses[i].generate_transfer(self.blockchain.size())
            if(mix == 0):
                self.blockchain.execute_transactions([i], [address], [amount])
            else:
                mixer_address = self.mixer.get_address()
                self.mixer.add_to_mixng_pool(i, address, amount)
                self.blockchain.execute_transactions([i], [mixer_address], [amount])
        mixer_address, addresses_out, sums_out = self.mixer.send_out()
        addresses_in = []
        for i in range(len(addresses_out)):
            addresses_in.append(mixer_address)
        self.blockchain.execute_transactions(addresses_in, addresses_out, sums_out)
            
    
    def simulate_random_exchange(self):
        
        for i in range(self.number_of_steps):
            self.step_with_multiple_addresses()
    
    def print_ledger(self):
        self.blockchain.print_ledger()
        
            

In [3]:
# User and Mixer with Multiple addresses

class mixer():
    
    def __init__(self, num_of_users, num_of_addresses):
        
        self.addresses_in = []
        self.addresses_out = []
        self.sums = []
        self.addresses = list(range(num_of_users, num_of_users+num_of_addresses))
        self.used_addresses = 0
    
    def add_address(self, public):
        self.addresses.append(public)
    
    def get_address(self):
        return self.addresses[self.used_addresses]
    
    def add_to_mixng_pool(self, address_in, address_out, sum_to_send):
        self.addresses_in.append(address_in)
        self.addresses_out.append(address_out)
        self.sums.append(sum_to_send)
    
    def send_out(self):
        
        incoming = self.addresses_in
        outcoming = self.addresses_in
        sums_out = self.sums

        if(len(self.sums) > 0):
            
            self.used_addresses += 1
            self.addresses_in = []
            self.addresses_out = []
            self.sums = []
            return self.addresses[self.used_addresses - 1], outcoming, sums_out
        
        self.addresses_in = []
        self.addresses_out = []
        self.sums = []
        
        return self.addresses[self.used_addresses], outcoming, sums_out
        
    

class user(address):
    
    def __init__(self, mixing_probability, mixer_address, *args, **kwargs):
        self.mixing_probability = mixing_probability 
        self.mixer_address = mixer_address
        super(user, self).__init__(*args, **kwargs)
        
    def generate_transfer(self, blockchain_size):
        probabilities = [1 - self.mixing_probability, self.mixing_probability]
        action = random.choices([0,1], weights = probabilities, k = 1) 
        amount_to_send = self.coins * random.uniform(0, 1)
        
        address_to_send = -1
        while(address_to_send == -1 or address_to_send == self.mixer_address):
            address_to_send = np.random.choice(blockchain_size, 1)
        
        return action[0], address_to_send[0], amount_to_send
     

In [4]:
# Measuring Ascendancy

def measure_development_capacity(network):
    N = network 
    Too = N.flatten().sum()
    D = 0
    column_sums = network.sum(axis=0)
    row_sums = network.sum(axis=1)
    x, y = network.shape
    for i in range(x):
        for j in range(y):
            d = network[i][j] / Too
            if(d > 0):
                D += network[i][j] * math.log10(d)

    return D
"""
def measure_average_mutual_information(network):
    
    N = network 
    Too = N.flatten().sum()
    AMI = 0
    column_sums = network.sum(axis=0)
    row_sums = network.sum(axis=1)
    x, y = network.shape
    for i in range(x):
        for j in range(y):
            a = network[i][j] * Too / (row_sums[i] * column_sums[j])
            if(a > 0):
                AMI += network[i][j] / Too * math.log10(a)

    return AMI
"""

def measure_ascendency(network):
    N = network 
    Too = N.flatten().sum()
    A = 0
    column_sums = network.sum(axis=0)
    row_sums = network.sum(axis=1)
    x, y = network.shape
    for i in range(x):
        for j in range(y):
            a = network[i][j] * Too / (row_sums[i] * column_sums[j])
            if(a > 0):
                A += network[i][j] * math.log10(a)

    return A

def measure_redundancy(network):
    N = network 
    Too = N.flatten().sum()
    R = 0
    column_sums = network.sum(axis=0)
    row_sums = network.sum(axis=1)
    x, y = network.shape
    for i in range(x):
        for j in range(y):
            r = network[i][j] * network[i][j] / (row_sums[i] * column_sums[j])
            if(r > 0):
                R += network[i][j] / Too * math.log10(r)

    return R